## Getting Reddit Comment Data

In [1]:
import praw
from praw.models import MoreComments
import tqdm
import pandas as pd

In [2]:
reddit = praw.Reddit(
    client_id="p1dG7hgoowK4BSlUdar1WQ",
    client_secret="pEePtSnw7KMDZi6fCzkKaOth_pgKpQ",
    password="outdoortuesday",
    user_agent="Big Data by u/DISWillJayminMaya ",
    username="DISWillJayminMaya ",
)

In [3]:
tickerlist = ['GME', 'Gamestop', 'SPY', 'TWTTR', 'Twitter', 'TSLA', 'Tesla', 'AMD']
subreddit = reddit.subreddit('wallstreetbets')

hot = subreddit.hot(limit=25) # getting first 15 posts in the 'hot' section of the subreddit
sum = [0] * len(tickerlist) # our output array
counttotal = 0 # total number of comment read
submissions_counter = 0

rel_comments = [] # List of comments that are relevant to the ticker list items

In [4]:
# Loop for fetching the comments and the amount of time each ticker is mentioned
for submissions in hot:
    if not submissions.stickied:
        submissions_counter+=1
        if submissions_counter > 5:
            comments = submissions.comments
            for comment in comments:
                if isinstance(comment, MoreComments):
                    continue
                counttotal+=1
                for i, ticker in enumerate(tickerlist):
                    if ticker in comment.body:
                        rel_comments.append(comment.body)
                        sum[i]=sum[i]+1

In [5]:
output=pd.DataFrame(data={'Tick': tickerlist, 'Counts': sum})
print('Total comments read: ',counttotal)
print(output[output['Counts']>0])

Total comments read:  1203
      Tick  Counts
0      GME       1
2      SPY       4
4  Twitter      50
5     TSLA       2
6    Tesla      13


## Getting Stock Information

In [7]:
import requests, os, json, csv

size = 'compact' # 'full' for complete historical data, 'compact' for most recent 100
ticker = ['GME', 'SPY', 'TWTTR', 'TSLA', 'AMD'] # stock tickers to search for
datatype = 'csv' # 'json' for JSON output, 'csv' for CSV output

for stock in ticker:
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={stock}&outputsize={size}&datatype={datatype}&apikey=QC1C7LRPUTLC597Q'
    response = requests.get(url)
    #Save CSV to file
    with open(f'{stock}.csv', 'wb') as file:
        file.write(response.content)

## Charting Data

In [8]:
amd_data = pd.read_csv("AMD.csv")
amd_data

,timestamp,open,high,low,close,volume
0,2022-04-25,89.860,91.37,88.6100,90.69,93481042
1,2022-04-22,90.030,91.46,87.9350,88.14,75017652
2,2022-04-21,95.160,96.23,89.2400,89.85,76680635
3,2022-04-20,97.840,97.91,93.2000,94.02,62322274
4,2022-04-19,93.385,97.07,92.8400,96.93,77069496
...,...,...,...,...,...,...
95,2021-12-07,143.900,145.76,141.0001,144.85,53359432
96,2021-12-06,141.135,141.31,134.2000,139.06,66776493
97,2021-12-03,151.645,152.38,140.7200,144.01,65910028
98,2021-12-02,147.675,152.53,146.4700,150.68,56161216


get avg, spread, median of sentiment of comments in any given period
    compare this with the performance of the stock

model:
    based on last x days of reddit comments, what is the price going to be?
    take data from x days, put it all into one vector, and predict from this
    
    

In [9]:

# https://medium.com/@pasdan/how-to-scrap-reddit-using-pushshift-io-via-python-a3ebcc9b83f4
import math
import json
import requests
import itertools
import numpy as np
import time
from datetime import datetime, timedelta

def make_request(uri, max_retries = 5):
    def fire_away(uri):
        response = requests.get(uri)
        assert response.status_code == 200
        return json.loads(response.content)
    current_tries = 1
    while current_tries < max_retries:
        try:
            time.sleep(1)
            response = fire_away(uri)
            return response
        except:
            time.sleep(1)
            current_tries += 1
    return fire_away(uri)


In [10]:
def pull_posts_for(subreddit, start_at, end_at):
    
    def map_posts(posts):
        return list(map(lambda post: {
            'id': post['id'],
            'created_utc': post['created_utc'],
            'prefix': 't4_'
        }, posts))
    
    SIZE = 500
    URI_TEMPLATE = r'https://api.pushshift.io/reddit/search/submission?subreddit={}&after={}&before={}&size={}'
    
    post_collections = map_posts(
        make_request(URI_TEMPLATE.format
                     (subreddit, start_at, end_at, SIZE))['data'])
    n = len(post_collections)
    while n == SIZE:
        last = post_collections[-1]
        new_start_at = last['created_utc'] - (10)
        
        more_posts = map_posts( \
            make_request( \
                URI_TEMPLATE.format( \
                    subreddit, new_start_at, end_at, bSIZE))['data'])
        
        n = len(more_posts)
        post_collections.extend(more_posts)
    return post_collections

In [11]:
subreddit = 'wallstreetbets'
end_at = math.ceil(datetime.utcnow().timestamp())
start_at = math.floor((datetime.utcnow() - timedelta(days=365)).timestamp())
posts = pull_posts_for(subreddit, start_at, end_at)
## ~ 4314
print(len(posts))
## ~ 4306
print(len(np.unique([ post['id'] for post in posts ])))

100
100


In [21]:
def give_me_intervals(start_at, number_of_days_per_interval = 3):
    
    end_at = math.ceil(datetime.utcnow().timestamp())
        
    ## 1 day = 86400,
    period = (86400 * number_of_days_per_interval)
    end = start_at + period
    yield (int(start_at), int(end))
    padding = 1
    while end <= end_at:
        start_at = end + padding
        end = (start_at - padding) + period
        yield int(start_at), int(end)
## test out the solution,
start_at = math.floor(\
     (datetime.utcnow() - timedelta(days=365)).timestamp())

In [22]:
# This is where the date is added. Could just make this a function and enter in dates and use that for indexing in the dataframes
def get_comments(start_date=(datetime.utcnow() - timedelta(days=7)), end_date=datetime.utcnow()):
    subreddit = 'wallstreetbets'
    start_at = math.floor(start_date.timestamp())
    posts = []
    for interval in give_me_intervals(start_at, 7):
        pulled_posts = pull_posts_for(
            subreddit, interval[0], interval[1])

        posts.extend(pulled_posts)
        time.sleep(.500)
        
    TIMEOUT_AFTER_COMMENT_IN_SECS = .350
    posts_from_reddit = []
    comments_from_reddit = []
    for submission_id in np.unique([ post['id'] for post in posts ]):
        submission = reddit.submission(id=submission_id)
        posts_from_reddit.append(submission)
        submission.comments.replace_more(limit=None)
        for comment in submission.comments.list():
            comments_from_reddit.append(comment)

            if TIMEOUT_AFTER_COMMENT_IN_SECS > 0:
                time.sleep(TIMEOUT_AFTER_COMMENT_IN_SECS)
    
    return comments_from_reddit

In [ ]:
data = get_comments()